In [ ]:
%load_ext autoreload
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
%autoreload 2

In [ ]:
pd.set_option('display.max_rows', 100)
SEED = 42
USE_nG = True # creting data only from nG subset

In [ ]:
# https://docs.google.com/spreadsheets/d/1MDleLmQ0Nlcg62x95e3xnkc5_j_i4IK_KQEHccDosG8/edit?usp=sharing
sheet_id = "1MDleLmQ0Nlcg62x95e3xnkc5_j_i4IK_KQEHccDosG8"
sheet_name = "clean"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

In [ ]:
df = pd.read_csv(url, 
                 header=0,
                 usecols=['patient', 'is_good', 'localization', 'comments'],
                 index_col=None, dtype={'patient':str,
                                        'is_good':str,
                                        'localization':str,
                                        'comments':str})

if USE_nG:
    df = df[df['patient'].apply(lambda x: x[0]=='n' or x[0]=='G')]
df_good = df.query('is_good == "1"')

In [ ]:
df_good['localization'].value_counts()

In [ ]:
manual_mapping = {'right tempor occipit':'occipit',
                  'left tempor occipit':'occipit',
                  'left occipit tempor':'occipit',
                  'left occipit':'occipit',
                  'right occipit':'occipit',
                  'left front tempor':'left tempor'
                 }

for k,v in manual_mapping.items():
    mask = df_good['localization'] == k
    df_good.loc[mask, 'localization'] = v

In [ ]:
df_good['localization'].value_counts()

In [ ]:
df_train, df_test = train_test_split(df_good, 
                                     stratify=df_good['localization'],
                                     random_state=SEED,
                                     test_size=10)

In [ ]:
df_train.shape, df_test.shape

In [ ]:
metadata = {'train':df_train['patient'].values,
            'test':df_test['patient'].values}
np.save('metadata/metadata_fcd_nG.npy', metadata)